In this scenario we want to continue our small datacenter from the basic
scenario by adding a Controller entity. This Controller will adjust the power
consumption of the computing system's nodes depending on the current power
delta.  We can easily create a new Controller by subclassing `Controller` from
[`vessim.controller`](https://github.com/dos-group/vessim/blob/main/vessim/controller.py).

```python
class Controller(ABC):
    def __init__(self, step_size: Optional[int] = None):
        self.step_size = step_size

    @abstractmethod
    def start(self, microgrid: Microgrid, clock: Clock, grid_signals: dict) -> None:
        """Supplies the controller with objects available after simulation start.

        Args:
            microgrid: The microgrid under control.
            clock: The clock of the simulation environment.
            grid_signals: All grid signals available in the simulation environment.
        """

    @abstractmethod
    def step(self, time: int, p_delta: float, actor_infos: dict) -> None:
        """Performs a simulation step.

        Args:
            time: Current simulation time.
            p_delta: Current power delta from the microgrid after the storage has been
                (de)charged. If negative, this power must be drawn from the public grid.
                If positive, the power can be fed to the public grid or must be curtailed.
            actor_infos: Contains the last "info" dictionaries by all actors in the
                microgrid. The info dictionary is defined by the actor and can contain
                any information about the actor's state.
        """

    def finalize(self) -> None:
        """This method can be overridden clean-up after the simulation finished."""
```

The most important aspect of this procedure is the implementation of the
`step()` method, which needs to be implemented as dictated by the Controller
ABC. The `step()` method is called every simulation step and allows a Controller
to act on the power delta from the Microgrid, the current simulation time and
the activity of the Actors. The `SimpleLoadBalancingController` only utilizes
the power delta and adjusts, depending on this value, the power consumption of
the computing system and in turn the power delta in the next simulation step.

In [ ]:
from __future__ import annotations
from _data import load_carbon_data, load_solar_data
from vessim.actor import ComputingSystem, Generator
from vessim.controller import Controller, Monitor
from vessim.cosim import Environment, Microgrid
from vessim.power_meter import MockPowerMeter
from vessim.signal import HistoricalSignal
from vessim.storage import SimpleBattery
from vessim.util import Clock


class SimpleLoadBalancingController(Controller):
    def __init__(self, max_load_adjustment: float, power_meters: list[MockPowerMeter]):
        super().__init__()
        self.max_load_adjustment = max_load_adjustment
        self.power_meters = power_meters

    def start(self, microgrid: Microgrid, clock: Clock, grid_signals: dict) -> None:
        pass

    def step(self, time: int, p_delta: float, actors: dict) -> None:
        # Calculate the maximum adjustment per MockPowerMeter
        adjustment_per_meter = min(abs(p_delta), self.max_load_adjustment) / len(self.power_meters)

        # Adjust the power setpoint for each MockPowerMeter
        for power_meter in self.power_meters:
            current_power = power_meter.measure()
            # Determine direction of adjustment
            if p_delta < 0:
                new_power = current_power + adjustment_per_meter
            else:
                new_power = max(0, current_power - adjustment_per_meter)
            power_meter.set_power(new_power)

Now we can add the controller to the basic scenario by instantiating it with a
reference to the power meters it can control and the maximum load adjustment for
each step. The rest of the scenario remains unchanged.

In [ ]:
environment = Environment(sim_start="2020-06-11 00:00:00")
environment.add_grid_signal("carbon_intensity", HistoricalSignal(load_carbon_data()))

power_meters: list = [
    MockPowerMeter(p=3),
    MockPowerMeter(p=7),
]
monitor = Monitor()  # Stores simulation result on each step
load_balancer = SimpleLoadBalancingController(max_load_adjustment=2, power_meters=power_meters)
microgrid = Microgrid(
    actors=[
        ComputingSystem(power_meters=power_meters),
        Generator(signal=HistoricalSignal(load_solar_data(sqm=0.4 * 0.5))),
    ],
    storage=SimpleBattery(capacity=1000, charge_level=500),
    controllers=[monitor, load_balancer],
    zone="DE",
    step_size=60,  # Global step size (can be overridden by actors or controllers)
)
environment.add_microgrid(microgrid)

environment.run(until=3600 * 24 * 2)  # Two days
monitor.to_csv("result_csv")

Now let's see how the load balancing controller affects our scenario. TODO show
nice plot and interpret a little